In [1]:
import numpy as np
import argparse
import matplotlib.pyplot as plt
import string
import re
import math
import os
import torch

In [2]:
thred_b = 0.005
alpha = 1.5
gap_time = 10
matrix_data_path = './data/matrix_data/'
valid_start = 8000 // gap_time
valid_end = 10000 // gap_time
test_start = 10000 // gap_time
test_end = 20000 // gap_time

valid_anomaly_score = np.zeros((valid_end - valid_start , 1))
test_anomaly_score = np.zeros((test_end - test_start, 1))

test_data_path = matrix_data_path + "test_data/"
reconstructed_data_path = matrix_data_path + "reconstructed_data/"
#reconstructed_data_path = matrix_data_path + "matrix_pred_data/"
criterion = torch.nn.MSELoss()

In [3]:
matrix_test_mse=[]
for i in range(valid_start, test_end):
	path_temp_1 = os.path.join(test_data_path, "test_data_" + str(i) + '.npy')
	gt_matrix_temp = np.load(path_temp_1)

	path_temp_2 = os.path.join(reconstructed_data_path, "reconstructed_data_" + str(i) + '.npy')
	#path_temp_2 = os.path.join(reconstructed_data_path, "pcc_matrix_full_test_" + str(i) + '_pred_output.npy')
	reconstructed_matrix_temp = np.load(path_temp_2)
	# reconstructed_matrix_temp = np.transpose(reconstructed_matrix_temp, [0, 3, 1, 2])
	#print(reconstructed_matrix_temp.shape)
	#first (short) duration scale for evaluation  
	select_gt_matrix = np.array(gt_matrix_temp)[-1][0] #get last step matrix

	select_reconstructed_matrix = np.array(reconstructed_matrix_temp)[0][0]

	#compute number of broken element in residual matrix
	select_matrix_error = np.square(np.subtract(select_gt_matrix, select_reconstructed_matrix))
	if i>=valid_end:
		matrix_test_mse.append(select_matrix_error)
	num_broken = len(select_matrix_error[select_matrix_error > thred_b])

	#print num_broken
	if i < valid_end:
		valid_anomaly_score[i - valid_start] = num_broken
	else:
		test_anomaly_score[i - test_start] = num_broken

In [4]:
valid_anomaly_max = np.max(valid_anomaly_score.ravel())
test_anomaly_score = test_anomaly_score.ravel()

In [5]:
import matplotlib.pyplot as plt

def plot_error_heatmap(error_matrix, timestep_index, save_dir=None):
    plt.figure(figsize=(4, 4))
    plt.imshow(error_matrix, cmap='jet')  # jet 色带突出异常值
    plt.colorbar()
    plt.title(f'Error Heatmap at t={timestep_index}')
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        plt.savefig(os.path.join(save_dir, f"error_heatmap_{timestep_index}.png"))
    else:
        plt.show()
    plt.close()

In [6]:
for i in range(len(matrix_test_mse)):
    if test_anomaly_score[i]>valid_anomaly_max * alpha:
        plot_error_heatmap(matrix_test_mse[i], i + test_start, save_dir='outputs/error_heatmaps')

In [30]:
anomaly_pos = np.zeros(5)
root_cause_gt = np.zeros((5, 3))
anomaly_span = [10, 30, 90]
root_cause_f = open("./data/test_anomaly.csv", "r")
row_index = 0
for line in root_cause_f:
	line=line.strip()
	anomaly_axis = int(re.split(',',line)[0])
	anomaly_pos[row_index] = anomaly_axis/gap_time - test_start - anomaly_span[row_index%3]/gap_time
	#print(anomaly_pos[row_index])
	root_list = re.split(',',line)[1:]
	for k in range(len(root_list)-1):
		root_cause_gt[row_index][k] = int(root_list[k])
	row_index += 1
root_cause_f.close()